<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/code/nlp_tutorial/NLP07%EF%BC%9A%E5%9F%BA%E4%BA%8ELSI%E7%9A%84%E6%96%87%E6%9C%AC%E7%9B%B8%E4%BC%BC%E5%BA%A6%E8%AE%A1%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
from collections import defaultdict
import jieba.posseg
import numpy as np
import codecs
import os
import pickle
from gensim import corpora,models,similarities

In [2]:
def tokenizer(filename, stop_words):
    """
    读取文件内容，并进行分词
    :param filename:文件名称
    :param stop_words:list,停用词
    :return:[[word1,word2]]
    """
    texts = []
    with open(filename, "r", encoding="utf-8") as f:
        for line in f.readlines():
            texts.append([token for token, _ in jieba.posseg.cut(line.rstrip()) if token not in stop_words])

    # 去除仅出现一次的单词
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [[token for token in text if frequency[token] > 1] for text in texts]

    return texts

In [4]:
stop_words_filepath = "/content/drive/My Drive/data/qa/data/stop_words.txt"
knowledge_texts_filepath = "/content/drive/My Drive/data/qa/data/knowledge.txt"
stop_words = codecs.open(stop_words_filepath, "r", encoding="utf-8").readlines()
stop_words = [w.strip() for w in stop_words]
texts = tokenizer(knowledge_texts_filepath, stop_words)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.250 seconds.
Prefix dict has been built successfully.


In [11]:
def topk_sim_ix(texts,stops,k):
    """
    :param file_name: 分词后的训练样本
    :param stop_words:停用词
    :param k:与每个文本top k相似度的文本
    :return:list
    """
    dictionary = corpora.Dictionary(texts) #构建词典
    corpus=[dictionary.doc2bow(text) for text in texts] #bow
    # 构建LSI模型
    lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=10)  # 潜在语义索引(分析),主题数量为10
    index = similarities.MatrixSimilarity(lsi[corpus],num_best=k)  # 计算相似度
    vec_lsi=lsi[corpus]
    return index[vec_lsi]


In [12]:
index=topk_sim_ix(texts,stop_words,5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


总共11740个文档，每个文档选择5个最相似的文档

In [15]:
len(index),len(texts),len(index[0])

(11740, 11740, 5)

第一个文档，除了文档本身外，最相似的就是第123、39、3985、11176个文档

In [16]:
index[0]

[(0, 1.0),
 (123, 0.9995017647743225),
 (39, 0.9973140954971313),
 (3985, 0.9969135522842407),
 (11176, 0.9967408776283264)]

In [18]:
for index_text in index[0]:
    print(texts[index_text[0]],index_text[1])

['地球', '宇宙', '中', '一颗', '行星', '运动', '规律'] 1.0
['地球', '自转', '照片', '中', '每颗', '星', '运动', '轨迹', '呈', '一段', '弧线'] 0.9995017647743225
['太阳系', '八颗', '行星', '中', '地球', '唯一', '一颗', '适合', '生物', '生存', '繁衍', '行星'] 0.9973140954971313
['式', '中', '已知', '水星', '金星', '地球', '火星', '木星', '土星', 'n', '值', '取'] 0.9969135522842407
['大气圈', '指', '包裹', '地球', '气体', '层', '气体', '中', '悬浮物', '厚度', '2000', '3000', '千米'] 0.9967408776283264
